# Environement Setup

In [3]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content


else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


Ambiente locale rilevato.


# Feature Extractor Setup

In [ ]:
import os

# CONFIGURAZIONE NOME CARTELLA
DIR_NAME = 'captain_cook_4d_gopro_resized' 

if IS_COLAB:
    # --- COLAB (Usa ancora lo zip per velocità) ---
    ZIP_VIDEOS = '/content/drive/MyDrive/AML_MistakeDetection_DATA/captain_cook_4d_gopro_resized.zip'
    DATA_ROOT = '/content/data'
    LOCAL_VIDEO_DIR = os.path.join(DATA_ROOT, DIR_NAME)
    
    if not os.path.exists(LOCAL_VIDEO_DIR):
        print(f"☁️ Colab: Estraggo video da {ZIP_VIDEOS}...")
        !mkdir -p {LOCAL_VIDEO_DIR}
        !unzip -q "{ZIP_VIDEOS}" -d "{LOCAL_VIDEO_DIR}"
else:
    # --- LOCALE (Usa la cartella già presente) ---
    DATA_ROOT = os.path.join(ROOT_DIR, 'data')
    LOCAL_VIDEO_DIR = os.path.join(DATA_ROOT, DIR_NAME)
    
    print(f"Cerco video in: {LOCAL_VIDEO_DIR}")

if os.path.exists(LOCAL_VIDEO_DIR):
    # Conta quanti mp4 ci sono (cerca anche nelle sottocartelle)
    num_videos = 0
    for root, dirs, files in os.walk(LOCAL_VIDEO_DIR):
        for file in files:
            if file.endswith('.mp4'):
                num_videos += 1
                
    if num_videos > 0:
        print(f"✅ Trovati {num_videos} video .mp4 pronti per l'estrazione.")
    else:
        print(f"⚠️ ATTENZIONE: La cartella '{LOCAL_VIDEO_DIR}' esiste ma NON contiene file .mp4!")
else:
    print(f"❌ ERRORE: La cartella '{LOCAL_VIDEO_DIR}' non esiste.")
    print(f"Assicurati di aver messo la tua cartella '{DIR_NAME}' dentro '{DATA_ROOT}'.")

🖥️ Modalità Locale.
📂 Cerco video in: c:\Users\enric\Desktop\MistakeDetection\data\captain_cook_4d_gopro_resized
✅ Trovati 22 video .mp4 pronti per l'estrazione.


In [ ]:
from utils.feature_extractor import PerceptionFeatureExtractor, ExtractionConfig

# 1. Configurazione
config = ExtractionConfig(
    video_dir = LOCAL_VIDEO_DIR,   # Cartella video scompattati
    output_root = DATA_ROOT,       # Dove salvare (creerà /perception_encoder/segment/1s)
    model_name = 'vit_pe_core_base_patch16_224.fb',
    batch_size = 64                # Riduci a 32 se la GPU va in errore (OOM)
)

# 2. Esecuzione
print("Avvio Perception Encoder Feature Extraction...")
extractor = PerceptionFeatureExtractor(config)
extractor.run()

Avvio Perception Encoder Feature Extraction...
🔄 Caricamento Modello: vit_pe_core_base_patch16_224.fb...


c:\Users\enric\Desktop\MistakeDetection\venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\enric\.cache\huggingface\hub\models--timm--vit_pe_core_base_patch16_224.fb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


✅ Modello pronto.
📁 Directory Output configurata: c:\Users\enric\Desktop\MistakeDetection\data\perception_encoder\segment\1s
🚀 Inizio estrazione su 22 video...


Estrazione Feature: 100%|██████████| 22/22 [03:46<00:00, 10.31s/it]

🎉 Estrazione Completata!


In [ ]:
import shutil
import os

ZIP_NAME = 'features_perception_encoder'

if IS_COLAB:
    # --- COLAB: SALVA SU DRIVE ---
    DRIVE_OUTPUT_DIR = '/content/drive/MyDrive/AML_MistakeDetection_DATA'
    OUTPUT_FULL_PATH = os.path.join(DRIVE_OUTPUT_DIR, ZIP_NAME + '.zip')
    
    print(f"☁️ Colab: Compressione e upload su Drive: {OUTPUT_FULL_PATH}")
    
    %cd {DATA_ROOT}
    !zip -r -q "{OUTPUT_FULL_PATH}" perception_encoder/
    
    print("✅ Zip salvato su Drive. Pronto per il training.")

else:
    # --- LOCALE: SALVA IN CARTELLA /DATA ---
    LOCAL_OUTPUT_DIR = os.path.join(ROOT_DIR, 'data')
    os.makedirs(LOCAL_OUTPUT_DIR, exist_ok=True)
    
    OUTPUT_BASE_PATH = os.path.join(LOCAL_OUTPUT_DIR, ZIP_NAME)
    
    print(f"🖥️ Locale: Compressione in corso verso: {LOCAL_OUTPUT_DIR} ...")
    
    shutil.make_archive(
        base_name=OUTPUT_BASE_PATH, 
        format='zip', 
        root_dir=DATA_ROOT, 
        base_dir='perception_encoder'
    )
    
    print(f"✅ Zip creato con successo: {OUTPUT_BASE_PATH}.zip")

🖥️ Locale: Compressione in corso verso: c:\Users\enric\Desktop\MistakeDetection\output ...
✅ Zip creato con successo: c:\Users\enric\Desktop\MistakeDetection\output\features_perception_encoder.zip
